In [1]:
from gurobipy import *
from random import uniform
import itertools as it
import time
import collections as coll



In [19]:
def get_contig_lengths(file):
    contig_len = {}
    for line in file:
        contig, length = line.split()
        length = int(length)
        if contig in contig_len:
            print("Error: double cont: " + contig + " | v1: " + str(contig_len[contig]) + " | v2: " + str(length))
        contig_len[contig] = length
    return contig_len

def wandle_datei_um(file, out_name, contig_len):
    out = open(out_name,'w')
    lines = []
    for line in file:
        a, b, dist = line.split()
        dist = float(dist) + contig_len[a]
        if a>b:
            a, b = b, a
            dist = - dist
        lines.append(a+'\t'+b+'\t'+str(dist)+'\n')
    lines.sort()
    out.writelines(lines)
    out.close()
#contig_len = get_contig_lengths(open('APDContigs.len'))
#wandle_datei_um(open('apd4.lst'), 'my_apd100.lst', contig_len)

In [35]:
def read_file(file_name, suffix = '_0'):
    """
    suffix -- wird dem Namen der Contigs angehangen, übergebe suffix = '' wenn der Suffix schon angehangen ist.
    """
    data = []
    with open(file_name) as file:
        for line in file:
            a, b, dist = line.split()
            dist = float(dist)
            data.append([a+suffix,b+suffix,dist])
    return data

def write_file(file_name, data):
    with open(file_name, 'w') as out:
        for a,b,dist in data:
            out.write(a +'\t'+ b +'\t'+str(dist) +'\n')
            
def write_sol(file_name, pos):
    pos = sorted(pos.items(), key = lambda x: x[1])
    with open(file_name, 'w') as out:
        for contig, position in pos:
            out.write(contig +'\t'+ str(position) + '\n')
            
            
def get_vars(data):
    contigs = set()
    for a, b, _ in data:
        contigs.add(a)
        contigs.add(b)
    return contigs

[['1001APD_0', '100APD_0', 53816.0],
 ['1001APD_0', '1046APD_0', 151296.0],
 ['1001APD_1', '10APD_0', 11235.0],
 ['1001APD_0', '10APD_0', 13986.0],
 ['1001APD_0', '10APD_0', 14388.0],
 ['1001APD_0', '10APD_0', 14548.0],
 ['1001APD_1', '1119APD_0', 17918.0],
 ['1001APD_0', '1119APD_0', 20720.0],
 ['1001APD_0', '1119APD_0', 21328.0],
 ['1001APD_0', '1130APD_0', 126362.0],
 ['1001APD_0', '1133APD_0', 135461.0],
 ['1001APD_0', '1298APD_0', -10059.0],
 ['1001APD_0', '1298APD_0', -10138.0],
 ['1001APD_0', '1298APD_0', -10191.0],
 ['1001APD_0', '1324APD_0', 10187.0],
 ['1001APD_0', '1324APD_0', 10294.0],
 ['1001APD_1', '1324APD_0', 7015.0],
 ['1001APD_0', '1324APD_0', 9771.0],
 ['1001APD_0', '1331APD_0', 38853.0],
 ['1001APD_0', '137APD_0', -33663.0]]

In [21]:
def init_model(model, data):

    # model.setParam(GRB.Param.LogToConsole, 0.0)
    
    # Variablen
    contig = model.addVars(get_vars(data))
    
    model.update()
        
    # Bedingungen
    fehler = []
    bedingung = []
    for a,b,dist in data:
        eps = model.addVar()
        c1 = model.addConstr( contig[b] - contig[a] - dist <= eps) # |(b - a) - dist| = fehler
        c2 = model.addConstr(-contig[b] + contig[a] + dist <= eps) #
        fehler.append(eps)
        bedingung.append([c1,c2])
        
    # Lösen
    model.setObjective(sum(fehler)/len(fehler), GRB.MINIMIZE)
    model.update()
    model.optimize()
    return contig, [fehler, bedingung]


In [28]:
def umsortieren(data, repeats, position, model, contig, bedingungen):
    fehler = coll.defaultdict(list)
    counter = 0
    for i, bedingung in enumerate(data):
        a, b, dist = bedingung
        
        contig_a = a[:a.find('_')]
        contig_b = b[:b.find('_')]
        
        # a und b werden zu den Repeat-Versionen umgeändert, die am besten
        # zu den Daten passen. Also zu denen Versionen a_i und b_j gemacht, 
        # für die b_j - a_i am nähsten an der vorgegebenen Distanz ist.
        new_a, new_b = min( 
            it.product(repeats[contig_a], repeats[contig_b]), 
            key = lambda contig: abs(position[contig[1]] - position[contig[0]] - dist)
        )
        bedingung[0] = new_a
        bedingung[1] = new_b
        res_dist = position[new_b] - position[new_a]
        
        fehler[new_a].append(res_dist - dist)
        fehler[new_b].append(dist - res_dist)
            
        if new_a != a or new_b != b:
            counter += 1
            model.remove(bedingungen[0][i])
            model.remove(bedingungen[1][i])
            eps = model.addVar()
            c1 = model.addConstr( contig[new_b] - contig[new_a] - dist <= eps) # |(b - a) - dist| = fehler
            c2 = model.addConstr(-contig[new_b] + contig[new_a] + dist <= eps) #
            bedingungen[0][i] = eps
            bedingungen[1][i] = [c1,c2]
            #print('\t'.join([a,b,str(dist),str(position[b] - position[a])]))
            #print('\t'.join([new_a, new_b,str(dist), str(position[new_b] - position[new_a])]))
            #print()
    print('Anzahl der Änderungen:',counter)
    print()
    return fehler


def extrahiere_daten(gruppe):
    # extrahiere Median und Länge der Gruppen und sortiere nach der Länge absteigend
    Gruppe = coll.namedtuple('Gruppe', ['min', 'max', 'median', 'anzahl'])
    return Gruppe(
        min = min(gruppe),
        max = max(gruppe),
        median = gruppe[len(gruppe)//2],
        anzahl = len(gruppe) 
    )
   
    
def gruppierung(fehler, min_abstand):
    sorted_fehler = sorted(fehler) #sorted([float(l.split()[2]) for l in data.splitlines()])
    # gruppiere sich stützende Daten
    previous = sorted_fehler[0]
    current_group = []
    groups = []
    for current in sorted_fehler:
        if current - previous > min_abstand:
            groups.append(extrahiere_daten(current_group))
            current_group = []

        current_group.append(current)
        previous = current
    groups.append(extrahiere_daten(current_group))
    groups.sort(key = lambda x: x.anzahl, reverse=True)
    return groups


def get_repeat(fehler, min_repeat_abstand = 500, min_güte = 0.01):
    max_güte = min_güte
    for contig in fehler:
        
        groups = gruppierung(fehler[contig], min_repeat_abstand)
        if len(groups) < 2:
            continue
            
        gruppe1, gruppe2 = groups[:2]
        
        rest = 0
        for gruppe in groups[2:]:
            rest += gruppe.anzahl


        entfernung = lambda x, y: max(y.min - x.max, x.min - y.max)
        calc_güte = ( lambda g1, g2, rest: 
                        g2.anzahl**2 * g1.anzahl
                        * ( entfernung(g1,g2) - min_repeat_abstand + 1 )**0.5
                        / ( g1.anzahl + rest**2 ) )
        
        güte = calc_güte(gruppe1, gruppe2, rest)/10000 
        if güte > max_güte:
            max_güte = güte
            repeat = contig, gruppe1, gruppe2
    print('güte', max_güte)
    print('gruppe1', repeat[1])
    print('gruppe2', repeat[2])
    print('rest', len(fehler[repeat[0]]) - repeat[1].anzahl - repeat[2].anzahl)
    print()
    return repeat


In [7]:

def verbessere_loop(datei, out): 
    
    data = read_file(datei)
    
    model = Model()
    model.setParam(GRB.Param.LogToConsole, 0)
    contigs, bedingungen = init_model(model, data)
    position = {contig: contigs[contig].X for contig in contigs}
    contig_repeat = {contig[:-2]: [contig] for contig in contigs}
        
    while True:
        # Lösen
        
        print('sortiere Bedingungen anderen repeats zu')
        fehler = umsortieren(data, contig_repeat, position, model, contigs, bedingungen)
        
        repeat, gruppe1, gruppe2 = get_repeat(fehler)
        
        contig = repeat[:repeat.rfind('_')]
        new_contig = contig +'_'+ str(len(contig_repeat[contig]))
        contig_repeat[contig].append(new_contig)
        
        print('alter Wert', position[repeat])
        position[new_contig] = position[repeat] + gruppe2.median
        position[repeat] = position[repeat] + gruppe1.median
        contigs[new_contig] = model.addVar()
        
        print('neuer Wert', position[repeat])
        print('       und', position[new_contig])
        
        print('ordne neuem repeat zu: ' + new_contig)
        umsortieren(data, contig_repeat, position, model, contigs, bedingungen)
        
        print()
        print()
        model.setObjective(sum(bedingungen[0])/len(bedingungen[0]), GRB.MINIMIZE)
        model.update()
        model.optimize()
        print('LP-Wert:',model.getObjective().getValue())
        print()
        
        write_file(out, data)

verbessere_loop('my_apd100.lst', 'out.lst' )

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 176.03321149012316
gruppe1 Gruppe(min=-278.0, max=11.0, median=-94.0, anzahl=149)
gruppe2 Gruppe(min=32440.0, max=32764.0, median=32627.0, anzahl=145)
rest 13

alter Wert 149257.0
neuer Wert 149163.0
       und 181884.0
ordne neuem repeat zu: 2328APD_1
Anzahl der Änderungen: 146



LP-Wert: 1000.2085081742397

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 191.6481235970143
gruppe1 Gruppe(min=-264.8114313739934, max=27.1885686260066, median=-80.8114313739934, anzahl=150)
gruppe2 Gruppe(min=32445.188568626007, max=32743.188568626007, median=32628.188568626007, anzahl=145)
rest 12

alter Wert 150212.811431374
neuer Wert 150132.0
       und 182841.0
ordne neuem repeat zu: 2185APD_1
Anzahl der Änderungen: 151



LP-Wert: 953.1540598284665

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 186.03397475651474
gruppe1 Gruppe(min=-267.0, max=107.0, median=-76.0, anzahl=171)
grupp

Anzahl der Änderungen: 0

güte 7.691348288187841
gruppe1 Gruppe(min=-94.25278433319181, max=321.0, median=-6.252784333191812, anzahl=100)
gruppe2 Gruppe(min=1710.7472156668082, max=3098.0, median=2531.0, anzahl=53)
rest 3

alter Wert 582136.0
neuer Wert 582129.7472156668
       und 584667.0
ordne neuem repeat zu: 688APD_1
Anzahl der Änderungen: 54



LP-Wert: 199.99258224512283

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 7.408142078045883
gruppe1 Gruppe(min=-111.25278433319181, max=2123.0, median=5.747215666808188, anzahl=239)
gruppe2 Gruppe(min=-1480.0, max=-1020.0, median=-1351.0, anzahl=61)
rest 2

alter Wert 399020.0
neuer Wert 399025.7472156668
       und 397669.0
ordne neuem repeat zu: 425APD_1
Anzahl der Änderungen: 62



LP-Wert: 199.2201928959357

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 6.22676921153613
gruppe1 Gruppe(min=-1356.0, max=352.0, median=22.0, anzahl=157)
gruppe2 Gruppe(min=-5617.0, max=-4189.0, median=-4928

LP-Wert: 165.51559432908996

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 1.4928088970210496
gruppe1 Gruppe(min=-1397.0, max=339.0, median=-7.0, anzahl=206)
gruppe2 Gruppe(min=4542.0, max=4862.747215666808, median=4693.0, anzahl=16)
rest 3

alter Wert 554120.0
neuer Wert 554113.0
       und 558813.0
ordne neuem repeat zu: 1897APD_1
Anzahl der Änderungen: 20



LP-Wert: 164.74419825570322

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 1.4514282182393818
gruppe1 Gruppe(min=-528.0, max=1092.0, median=2.0, anzahl=103)
gruppe2 Gruppe(min=-4626.0, max=-4496.0, median=-4587.0, anzahl=16)
rest 2

alter Wert 170857.0
neuer Wert 170859.0
       und 166270.0
ordne neuem repeat zu: 1408APD_1
Anzahl der Änderungen: 17



LP-Wert: 163.97961026019635

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 1.3340959952134046
gruppe1 Gruppe(min=-267.0, max=878.0, median=14.0, anzahl=99)
gruppe2 Gruppe(min=-4189.0, max=-3998.0, median=-4

UnboundLocalError: local variable 'repeat' referenced before assignment

In [36]:
def verbessere_loop(datei, out): 
    
    data = read_file(datei,'')
    
    model = Model()
    model.setParam(GRB.Param.LogToConsole, 0)
    contigs, bedingungen = init_model(model, data)
    print('LP-Wert:',model.getObjective().getValue())
    print()
    position = {contig: contigs[contig].X for contig in contigs}
    contig_repeat = coll.defaultdict(list)
    for contig in contigs:
        contig_repeat[contig[:-2]].append(contig)
        
    while True:
        # Lösen
        
        print('sortiere Bedingungen anderen repeats zu')
        fehler = umsortieren(data, contig_repeat, position, model, contigs, bedingungen)
        
        repeat, gruppe1, gruppe2 = get_repeat(fehler)
        
        contig = repeat[:repeat.rfind('_')]
        new_contig = contig +'_'+ str(len(contig_repeat[contig]))
        contig_repeat[contig].append(new_contig)
        
        print('alter Wert', position[repeat])
        position[new_contig] = position[repeat] + gruppe2.median
        position[repeat] = position[repeat] + gruppe1.median
        contigs[new_contig] = model.addVar()
        
        print('neuer Wert', position[repeat])
        print('       und', position[new_contig])
        
        print('ordne neuem repeat zu: ' + new_contig)
        umsortieren(data, contig_repeat, position, model, contigs, bedingungen)
        
        print()
        print()
        model.setObjective(sum(bedingungen[0])/len(bedingungen[0]), GRB.MINIMIZE)
        model.update()
        model.optimize()
        print('LP-Wert:',model.getObjective().getValue())
        print()
        
        write_file(out, data)
        write_sol('sol2.lst', position)

verbessere_loop('out.lst', 'out2.lst' )

LP-Wert: 150.80592862174163

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 15

güte 0.9959454134439298
gruppe1 Gruppe(min=-525.0, max=946.0, median=-21.0, anzahl=164)
gruppe2 Gruppe(min=5285.0, max=5530.0, median=5483.0, anzahl=14)
rest 6

alter Wert 399513.6353455794
neuer Wert 399492.6353455794
       und 404996.6353455794
ordne neuem repeat zu: 381APD_1
Anzahl der Änderungen: 20



LP-Wert: 149.77930530108526

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 1.0088441898709086
gruppe1 Gruppe(min=-678.0, max=447.0, median=5.0, anzahl=158)
gruppe2 Gruppe(min=-2986.0, max=-2749.7088780517806, median=-2780.0, anzahl=16)
rest 1

alter Wert 441833.6353455794
neuer Wert 441838.6353455794
       und 439053.6353455794
ordne neuem repeat zu: 902APD_1
Anzahl der Änderungen: 17



LP-Wert: 149.3399636199448

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 0.9824987811299692
gruppe1 Gruppe(min=-464.0, max=433.0, median=-3.0, anzahl=18

Anzahl der Änderungen: 17



LP-Wert: 138.88246261531077

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 0.5280832699562787
gruppe1 Gruppe(min=-21.530515780577844, max=44.0, median=0.0, anzahl=35)
gruppe2 Gruppe(min=1576.0, max=1738.0, median=1613.0, anzahl=13)
rest 1

alter Wert 65616.0
neuer Wert 65616.0
       und 67229.0
ordne neuem repeat zu: 1993APD_1
Anzahl der Änderungen: 13



LP-Wert: 138.67730775985768

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 0.5159699641584259
gruppe1 Gruppe(min=-71.0, max=213.0, median=0.0, anzahl=80)
gruppe2 Gruppe(min=2301.0, max=2651.0, median=2438.0, anzahl=12)
rest 3

alter Wert 777001.0
neuer Wert 777001.0
       und 779439.0
ordne neuem repeat zu: 721APD_1
Anzahl der Änderungen: 12



LP-Wert: 138.40045714278762

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 0.5063689070233283
gruppe1 Gruppe(min=-64.0, max=580.0, median=6.0, anzahl=31)
gruppe2 Gruppe(min=-1712.0, max=-87

LP-Wert: 129.5827988694946

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 0.2743348156428135
gruppe1 Gruppe(min=-228.0, max=29.0, median=-35.0, anzahl=21)
gruppe2 Gruppe(min=1346.0, max=1732.0, median=1620.0, anzahl=13)
rest 4

alter Wert 750349.0
neuer Wert 750314.0
       und 751969.0
ordne neuem repeat zu: 1753APD_1
Anzahl der Änderungen: 15



LP-Wert: 129.35644166233035

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 0.27423085165604544
gruppe1 Gruppe(min=-43.0, max=78.0, median=4.0, anzahl=27)
gruppe2 Gruppe(min=-2453.0, max=-2378.0, median=-2426.0, anzahl=8)
rest 0

alter Wert 54182.0
neuer Wert 54186.0
       und 51756.0
ordne neuem repeat zu: 93APD_1
Anzahl der Änderungen: 8



LP-Wert: 129.1659434263186

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 0.2696636460027755
gruppe1 Gruppe(min=-193.0, max=220.0, median=24.0, anzahl=34)
gruppe2 Gruppe(min=-6363.0, max=-5390.0, median=-6015.611033616122, anzahl=

güte 0.16052360691747558
gruppe1 Gruppe(min=-168.0, max=38.11312446185548, median=-1.0, anzahl=82)
gruppe2 Gruppe(min=2070.0, max=2276.0, median=2252.0, anzahl=7)
rest 4

alter Wert 75242.0
neuer Wert 75241.0
       und 77494.0
ordne neuem repeat zu: 61APD_1
Anzahl der Änderungen: 11



LP-Wert: 123.21929381217633

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 0.15764661675588915
gruppe1 Gruppe(min=-213.0, max=209.0, median=-2.0, anzahl=85)
gruppe2 Gruppe(min=1705.0, max=1800.0, median=1756.0, anzahl=11)
rest 11

alter Wert 780256.0
neuer Wert 780254.0
       und 782012.0
ordne neuem repeat zu: 552APD_1
Anzahl der Änderungen: 14



LP-Wert: 122.97746600939715

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 0.20661043329552772
gruppe1 Gruppe(min=-211.0, max=211.0, median=0.0, anzahl=85)
gruppe2 Gruppe(min=-2635.0, max=-2530.0, median=-2575.0, anzahl=7)
rest 1

alter Wert 780254.0
neuer Wert 780254.0
       und 777679.0
ordne neuem repeat 

LP-Wert: 117.62189789420812

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 0.11941923627730816
gruppe1 Gruppe(min=-71.0, max=329.0, median=0.0, anzahl=71)
gruppe2 Gruppe(min=-7507.0, max=-7088.0, median=-7211.0, anzahl=5)
rest 7

alter Wert 838545.0
neuer Wert 838545.0
       und 831334.0
ordne neuem repeat zu: 479APD_2
Anzahl der Änderungen: 11



LP-Wert: 117.02449915032807

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 0.11892262485133183
gruppe1 Gruppe(min=-190.0, max=115.0, median=-14.0, anzahl=111)
gruppe2 Gruppe(min=1725.0, max=1844.0, median=1833.0, anzahl=6)
rest 1

alter Wert 223262.0
neuer Wert 223248.0
       und 225095.0
ordne neuem repeat zu: 1333APD_1
Anzahl der Änderungen: 7



LP-Wert: 116.90019273620382

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 0.11847227523771121
gruppe1 Gruppe(min=-372.0, max=1159.0, median=9.0, anzahl=135)
gruppe2 Gruppe(min=-2065.0, max=-1954.0, median=-1996.0, anzahl=

...

Anzahl der Änderungen: 0

güte 0.01463693957082559
gruppe1 Gruppe(min=-143.03497768891975, max=194.96502231108025, median=-4.034977688919753, anzahl=12)
gruppe2 Gruppe(min=22117.96502231108, max=22117.96502231108, median=22117.96502231108, anzahl=1)
rest 0

alter Wert 801739.0349776889
neuer Wert 801735.0
       und 823857.0
ordne neuem repeat zu: 146APD_3
Anzahl der Änderungen: 4



LP-Wert: 96.63380957805911

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 0.013621967718914453
gruppe1 Gruppe(min=-847.0, max=804.0, median=0.0, anzahl=41)
gruppe2 Gruppe(min=2520.0, max=2527.0, median=2527.0, anzahl=2)
rest 1

alter Wert 118993.0
neuer Wert 118993.0
       und 121520.0
ordne neuem repeat zu: 529APD_1
Anzahl der Änderungen: 3



LP-Wert: 96.55896659304359

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 0.013620900651543612
gruppe1 Gruppe(min=-360.0, max=324.0, median=0.0, anzahl=85)
gruppe2 Gruppe(min=2010.0, max=2044.0, median=2044.0, anzah

Anzahl der Änderungen: 3



LP-Wert: 94.89688578363338

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 0.011597510894720817
gruppe1 Gruppe(min=-110.0, max=57.0, median=0.0, anzahl=32)
gruppe2 Gruppe(min=1450.0, max=1495.0, median=1495.0, anzahl=2)
rest 1

alter Wert 43619.0
neuer Wert 43619.0
       und 45114.0
ordne neuem repeat zu: 1743APD_2
Anzahl der Änderungen: 3



LP-Wert: 94.86732245837737

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 0.011595688854052613
gruppe1 Gruppe(min=-182.0, max=24.0, median=0.0, anzahl=8)
gruppe2 Gruppe(min=-937.0, max=-847.0, median=-931.0, anzahl=3)
rest 0

alter Wert 77494.0
neuer Wert 77494.0
       und 76563.0
ordne neuem repeat zu: 61APD_2
Anzahl der Änderungen: 3



LP-Wert: 94.84243743785422

sortiere Bedingungen anderen repeats zu
Anzahl der Änderungen: 0

güte 0.011131154294068848
gruppe1 Gruppe(min=-337.0, max=117.0, median=0.0, anzahl=69)
gruppe2 Gruppe(min=-1717.0, max=-1633.0, median=-1633.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Gamer-PC\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-36-ea4f0b08573b>", line 49, in <module>
    verbessere_loop('out.lst', 'out2.lst' )
  File "<ipython-input-36-ea4f0b08573b>", line 43, in verbessere_loop
    print('LP-Wert:',model.getObjective().getValue())
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Gamer-PC\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2039, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Gamer-PC\Miniconda3\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_reco

KeyboardInterrupt: 

In [25]:
def verbessere_loop(datei, out): 
    
    data = read_file(datei,'')
    
    model = Model()
    #model.setParam(GRB.Param.LogToConsole, 0)
    contigs, bedingungen = init_model(model, data)
    print('LP-Wert:',model.getObjective().getValue())
    print()
    position = {contig: contigs[contig].X for contig in contigs}
    contig_repeat = coll.defaultdict(list)
    for contig in contigs:
        contig_repeat[contig[:-2]].append(contig)
        
    for i in range(2):
        # Lösen
        
        print('sortiere Bedingungen anderen repeats zu')
        fehler = umsortieren(data, contig_repeat, position, model, contigs, bedingungen)
        
        repeat, gruppe1, gruppe2 = get_repeat(fehler)
        
        contig = repeat[:repeat.rfind('_')]
        new_contig = contig +'_'+ str(len(contig_repeat[contig]))
        contig_repeat[contig].append(new_contig)
        
        print('alter Wert', position[repeat])
        position[new_contig] = position[repeat] + gruppe2.median
        position[repeat] = position[repeat] + gruppe1.median
        contigs[new_contig] = model.addVar()
        
        print('neuer Wert', position[repeat])
        print('       und', position[new_contig])
        
        print('ordne neuem repeat zu: ' + new_contig)
        umsortieren(data, contig_repeat, position, model, contigs, bedingungen)
        
        print()
        print()
        model.setObjective(sum(bedingungen[0])/len(bedingungen[0]), GRB.MINIMIZE)
        model.update()
        model.optimize()
        print('LP-Wert:',model.getObjective().getValue())
        print()
        
        write_file(out, data)

verbessere_loop('out.lst', 'out4.lst' )

Optimize a model with 202210 rows, 103091 columns and 606630 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 1e-05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+05]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 13 rows and 16 columns
Presolve time: 0.68s
Presolved: 202197 rows, 103075 columns, 606590 nonzeros

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 1082
 AA' NZ     : 3.860e+06
 Factor NZ  : 1.080e+07 (roughly 200 MBytes of memory)
 Factor Ops : 4.617e+09 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   7.52190158e+05  0.00000000e+00  7.52e+05 0.00e+00  7.51e+04     3s
   1   7.60408557e+05 -9.93032983e+03  1.75e-10 9.33e-03  7.32e+03     4s
   2   7.60387264e+05  3.44482404e+01  1.16e-10 7.08e-07  3.06e+00     4s
   3   7


Barrier statistics:
 Dense cols : 1082
 AA' NZ     : 3.860e+06
 Factor NZ  : 1.080e+07 (roughly 200 MBytes of memory)
 Factor Ops : 4.617e+09 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   7.52190158e+05  0.00000000e+00  7.52e+05 0.00e+00  7.51e+04     3s
   1   7.60408556e+05 -9.93011830e+03  2.33e-10 9.33e-03  7.32e+03     4s
   2   7.60387263e+05  3.44493011e+01  4.66e-10 7.08e-07  3.06e+00     4s
   3   7.40584484e+04  1.61924293e+01  5.82e-11 2.91e-07  2.48e-01     5s
   4   3.94299248e+03  1.87899820e+01  1.75e-10 1.64e-07  1.29e-02     6s
   5   3.90744167e+02  4.88710463e+01  8.73e-11 3.32e-08  1.13e-03     7s
   6   2.28450833e+02  1.03538734e+02  8.13e-12 6.60e-09  4.11e-04     8s
   7   1.76818389e+02  1.24451388e+02  8.99e-11 2.86e-09  1.72e-04     9s
   8   1.62793917e+02  1.32623781e+02  3.69e-11 1.80e-09  9.94e-05    10s
   9   1.5738

  22   1.49397065e+02  1.49079561e+02  7.63e-11 2.25e-11  1.05e-06    20s
  23   1.49380257e+02  1.49139319e+02  5.16e-12 1.73e-11  7.95e-07    21s
  24   1.49366683e+02  1.49179395e+02  3.86e-11 1.38e-11  6.18e-07    22s
  25   1.49358966e+02  1.49215192e+02  5.88e-11 1.07e-11  4.74e-07    23s
  26   1.49353431e+02  1.49270730e+02  5.57e-11 6.14e-12  2.73e-07    23s
  27   1.49347614e+02  1.49293708e+02  4.96e-11 4.20e-12  1.78e-07    24s
  28   1.49345422e+02  1.49309676e+02  3.09e-11 2.82e-12  1.18e-07    25s
  29   1.49344337e+02  1.49315365e+02  4.82e-11 2.29e-12  9.56e-08    26s
  30   1.49342886e+02  1.49322728e+02  2.59e-11 2.30e-12  6.65e-08    26s
  31   1.49341925e+02  1.49326326e+02  3.43e-11 2.10e-12  5.15e-08    27s
  32   1.49341073e+02  1.49328849e+02  1.93e-11 1.77e-12  4.04e-08    28s
  33   1.49340260e+02  1.49333807e+02  2.52e-11 1.07e-12  2.13e-08    29s
  34   1.49340005e+02  1.49339481e+02  5.61e-11 2.04e-13  1.73e-09    30s
  35   1.49339969e+02  1.49339959e+02 

In [28]:

datei = 'mytest1000.lst'
out = 'test_out.lst'

data = read_file(datei)

var = get_vars(data)

# Setup Model
model = Model("Gen")
# model.setParam("TimeLimit",30)
# model.setParam(GRB.Param.LogToConsole, 0.0)
# model.Params.OptimalityTol = 1e-2

# Variablen
contig = model.addVars(var)
for v, start in var.items():
   # print(v)
    contig[v].start = start

fehler = [
    model.addVar(name = '\t'.join([a, b, str(dist)]))
    for a,b,dist in data
]
# for a,b,dist in data:
#    print('\t'.join([a, b, str(dist)]))
model.update()

# Bedingungen
anz = 0
for bedingung in fehler:
    anz += 1
    a,b,dist = bedingung.VarName.split()
    dist = float(dist)
    model.addConstr(contig[b] - contig[a]  - dist <= bedingung)  # |(b - a) - dist| = fehler
    model.addConstr(-contig[b] + contig[a]  + dist <= bedingung) #

# Lösen
model.setObjective(sum(fehler)/anz, GRB.MINIMIZE)
model.update()
model.optimize()

model.setObjective((sum(fehler)+fehler[30]+fehler[300])/(anz+2), GRB.MINIMIZE)
model.optimize()


Optimize a model with 31704 rows, 17675 columns and 95112 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-05, 6e-05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 3e+05]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 127 rows and 127 columns
Presolve time: 0.15s
Presolved: 31577 rows, 17548 columns, 94730 nonzeros

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 111
 AA' NZ     : 5.370e+05
 Factor NZ  : 7.599e+05 (roughly 26 MBytes of memory)
 Factor Ops : 2.663e+07 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.28651495e+05  0.00000000e+00  5.31e+05 0.00e+00  5.28e+04     0s
   1   5.33234367e+05 -2.91981265e+04  4.66e-10 1.01e-02  5.43e+03     1s
   2   5.31027312e+05 -6.45349177e+01  3.73e-09 6.38e-06  1.43e+01     1s
   3   6.56926

In [9]:
contig, rest, gruppe1, gruppe2, _güte = get_repeat(fehler)
        
f.seek(0)
new_name = 'my_apd' + str(int(datei[6:-4]) + 1) + '.lst'
verdopple_contig2(f, new_name, mcontig, contig, gruppe1, gruppe2, rest)
#verdopple_contig(f,new_name,contig,mcontig, mean)
f.close()
datei = new_name

Teile 2328APD auf:
1. Teil von -278 bis12
2. Teil von 32439 bis32765
und 13 auf beide
2328APD	1004APD	-65.0
2328APD	1004APD	-73.0
2328APD	1004APD	-32783.0
2328APD	1004APD	32564.0
2328APD	1004APD	32693.0
2328APD	1013APD	-107.0
2328APD	1013APD	32647.0
2328APD	1028APD	-105.0
2328APD	1028APD	32649.0
2328APD	107APD	-94.0
2328APD	107APD	32624.0
2328APD	108APD	-92.0
2328APD	108APD	32531.0
2328APD	108APD	32647.0
2328APD	1094APD	32625.0
2328APD	1094APD	-93.0
2328APD	1095APD	32626.0
2328APD	1095APD	-92.0
2328APD	111APD	32625.0
2328APD	111APD	-93.0
2328APD	1120APD	32647.0
2328APD	1120APD	-107.0
2328APD	1129APD	-94.0
2328APD	1129APD	32624.0
2328APD	115APD	-92.0
2328APD	115APD	32626.0
2328APD	1178APD	32647.0
2328APD	1178APD	-107.0
2328APD	1190APD	32647.0
2328APD	1190APD	-107.0
2328APD	1214APD	-105.0
2328APD	1214APD	32649.0
2328APD	1257APD	32633.0
2328APD	1257APD	-85.0
2328APD	1294APD	32627.0
2328APD	1294APD	-91.0
2328APD	1325APD	32624.0
2328APD	1325APD	-94.0
2328APD	135APD	32627.0
2328APD	135APD	-9

In [142]:

def grouping(fehler, eps = 1000):
    sorted_dist_errors = sorted(fehler) #sorted([float(l.split()[2]) for l in data.splitlines()])
    last = sorted_dist_errors[0]
    current_group = []
    groups = []
    for current in sorted_dist_errors:
        if current-last > eps:
            groups.append(current_group)
            current_group = []

        current_group.append(current)
        last = current
    groups.append(current_group)
    
    group_mean_anz = [
        (
            int(min(group)), # min
            int(max(group)+1), # max
            len(group) # len
        ) 
        for group in groups
    ]
    group_mean_anz.sort(key = lambda x: x[2],reverse=True)
    erg = [0]
    for group in group_mean_anz[2:]:
        erg[0] += group[2]
    erg += group_mean_anz[:2]
    #    erg[2][1] += group[2]
    #erg.append(group_mean_anz[2:])
    return erg

In [10]:
%store

Stored variables and their in-db values:
a                   -> 0
fehler              -> {'1001APD': [0.0, 0.0, 2762.0, 11.0, -391.0, -551.
mcontig             -> '<unavailable>'
meps                -> '<unavailable>'


In [151]:
for i in range(20):
    group = grouping(fehler[wert[i][0]],100)
    print(group[0],group[1][2],group[2][2])
    # group = sorted(group,key = lambda x: x[2][2],reverse=True)
    # group2 = group[:5]
    #out = ''
    #for g in group2:
    #    out += str(g[0]) + ' : ' + str(g[1]) + '\t'
    #out = str(group2[1][1])

19 139 99
24 124 68
15 11 11
35 108 67
16 77 21
18 99 24
26 147 145
13 149 145
13 149 145
17 167 144
17 155 131
20 160 132
19 161 132
19 160 131
19 161 131
18 162 131
17 163 131
24 161 131
1 8 1
1 9 1
9 10 4
0 4 1
6 47 8
20 212 113
0 13 4
1 2 1
6 7 3
1 11 1
1 4 1
5 3 3
2 11 5
24 131 131
2 12 2
15 25 5
2 12 2
8 10 5
0 2 1
1 12 2
5 98 2
3 18 5
5 97 3
8 94 4
5 97 3
5 98 3
5 98 3
3 18 1
51 123 27
27 134 48
0 1 1
6 15 9
22 12 6
3 23 1
27 68 25
4 18 5
13 34 7
3 24 1
3 24 1
3 24 1
5 150 15
13 103 13
13 41 21
3 22 1
3 25 1
3 25 1
3 25 1
3 25 1
10 31 3
4 149 15
4 149 15
2 144 15
3 93 2
1 131 14
13 119 9
6 67 2
8 24 6
5 43 28
1 131 13
2 37 33
5 77 8
5 20 11
22 145 15
17 50 12
7 133 11
1 69 1
3 68 2
18 27 8
1 70 1
3 133 11
2 69 2
8 132 11
6 76 2
2 131 11
2 131 11
2 131 11
2 131 11
2 131 11
2 131 11
2 132 11
3 131 11
2 132 11
2 132 11
0 18 9
2 132 11
5 131 11
2 133 11
2 134 11
4 132 11
4 132 11
2 134 11
14 32 3
2 135 11
28 84 34
4 135 11
4 135 11
2 135 11
2 135 11
5 71 8
7 133 11
10 35 2
14 34 4
4

IndexError: list index out of range

In [165]:
def get_analystics(file, contigs):
    summe = {}
    anzahl = {}
    fehler = {}
    for line in file:
        a, b, dist = line.split()
        dist = float(dist)
        res_dist = contigs[b].X - contigs[a].X
        summe.setdefault(a, 0)
        summe[a] += abs(dist - res_dist)
        anzahl.setdefault(a, 0)
        anzahl[a] += 1
        fehler.setdefault(a, []).append(res_dist - dist)
        summe.setdefault(b, 0)
        summe[b] += abs(dist - res_dist)
        anzahl.setdefault(b, 0)
        anzahl[b]  += 1
        fehler.setdefault(b, []).append(dist - res_dist)
    return fehler, summe, anzahl

f = open(datei)
fehler, summe, anzahl = get_analystics(f, mcontig)
wert = [(f, summe[f]/anzahl[f]) for f in summe]
wert.sort(key = lambda x: x[1],reverse=True)

In [144]:
contig = wert[0][0]
mean = sum(fehler[contig])/anzahl[contig]
fehler[contig]

[12756.724722845995,
 12756.724722845995,
 0.0,
 0.0,
 48.0,
 48.0,
 -15491.944657318918,
 -15491.944657318918]

In [104]:
def verdopple_contig(file, out_name, contig, contigs, mean, sicherheit = 1000):
    out = open(out_name,'w')
    z1 = z2 = 0
    m1 = m2 = set()
    print('Teile '+ contig + ' auf bei Differenz: ' + str(mean))
    for line in file:
        a, b, dist = bedingung = line.split()
        if not contig in bedingung:
            out.write(line)
            continue
        dist = float(dist)
        if contig == b:
            a, b = b, a
            dist = - dist
            
        res_dist = contigs[b].X - contigs[a].X
        
        print(a + '\t' + b + '\t' + str(res_dist - dist))
        
        index = bedingung.index(contig)
        if res_dist - dist < mean - sicherheit:
            bedingung[index] = contig + '_1'
            m1.add(bedingung[1-index])
            z1 += 1
        elif res_dist - dist > mean + sicherheit:
            bedingung[index] = contig + '_2'
            m2.add(bedingung[1-index])
            z2 += 1
        else:
            print('konnte Bedingung nicht klar zuordnen: ' + line)
            raise
        out.write(bedingung[0]+'\t'+bedingung[1]+'\t'+bedingung[2]+'\n')
    if len(m1) < 2 or len(m2) < 2 or z1 < 4 or z2 < 4:
        print('zu wenig Daten' + line)
        raise
new_name = 'my_apd' + str(int(datei[6:-4]) + 1) + '.lst'
verdopple_contig(open(datei),new_name,contig,mcontig, mean)
datei = new_name

Teile 339APD auf bei Differenz: 3256.6027397260273
339APD	1004APD_2	6541.0
339APD	1004APD_1	-11.0
339APD	1004APD_1	6541.0
339APD	1004APD_2	-11.0
339APD	1013APD	-21.0
339APD	1013APD	6531.0
339APD	1028APD	-19.0
339APD	1028APD	6533.0
339APD	107APD	6533.0
339APD	107APD	-12.0
339APD	108APD	-21.0
339APD	108APD	6531.0
339APD	1094APD	6535.0
339APD	1094APD	-10.0
339APD	1095APD	-10.0
339APD	1095APD	6535.0
339APD	111APD	-10.0
339APD	111APD	6535.0
339APD	1120APD	-19.0
339APD	1120APD	6533.0
339APD	1129APD	6534.0
339APD	1129APD	-11.0
339APD	115APD	6535.0
339APD	115APD	-10.0
339APD	1178APD	-21.0
339APD	1178APD	6531.0
339APD	1190APD	-19.0
339APD	1190APD	6533.0
339APD	1214APD	-19.0
339APD	1214APD	6533.0
339APD	1257APD	6540.0
339APD	1257APD	-5.0
339APD	1294APD	-9.0
339APD	1294APD	6536.0
339APD	1325APD	6534.0
339APD	1325APD	-11.0
339APD	135APD	-8.0
339APD	135APD	6537.0
339APD	1372APD	6533.0
339APD	1372APD	-12.0
339APD	1407APD	6535.0
339APD	1407APD	-10.0
339APD	1419APD	-18.0
339APD	1419APD	6534.0
339APD	1

In [167]:
for i in range(10):
    z = sorted(fehler[wert[i][0]])

    l = []
    a = z[0]
    za = []
    lz = []
    for b in z:
        if b-a > 100:
            lz.append(za)
            za = []

        a = b
        za.append(b)
    newz = [(int(sum(za)/len(za)),len(za)) for za in lz]
    print(newz)


[(-15491, 2), (24, 4)]
[(-24954, 2), (4, 7)]
[(-27921, 1), (-22259, 1), (-9750, 1), (-1620, 6), (20, 13), (5686, 2), (13215, 1)]
[(-12756, 2)]
[(-1653, 6), (-11, 6), (5656, 6), (13187, 1)]
[(0, 2)]
[(-13201, 2), (-7212, 3), (-5652, 9), (81, 17), (301, 3)]
[(-12487, 1), (-5233, 1), (-2977, 1), (-884, 1), (-509, 9), (-2, 231), (308, 1), (500, 2), (859, 50), (1259, 1), (1810, 1), (4403, 31), (6642, 1), (9097, 1), (9263, 1)]
[(5, 145), (2717, 2), (2963, 2), (4708, 1), (27558, 16)]
[(-13765, 1), (-8653, 1), (-8528, 1), (-3298, 17), (-1663, 1), (-102, 7), (236, 4), (504, 18), (9411, 4), (10606, 1)]


In [171]:
wert[3][0]
fehler[wert[7][0]]
contig_len = get_contig_lengths(open('APDContigs.len'))
wert[7][0]

'58APD'

In [145]:
for i in range(20):
    [print(f) for f in fehler[wert[i][0]]]
    print("-------------------------------------------")

12756.724722845995
12756.724722845995
0.0
0.0
48.0
48.0
-15491.944657318918
-15491.944657318918
-------------------------------------------
43.57991052826401
43.57991052826401
-24954.271385183318
-24954.271385183318
-62.0
9750.0
4.0
4.0
0.0
0.0
-------------------------------------------
5686.0
5686.0
-22259.0
13215.0
39.57991052826401
39.57991052826401
-1602.0
-1602.0
33.0
33.0
24954.271385183318
24954.271385183318
62.0
-9750.0
-1632.0
-1632.0
0.0
0.0
-4.0
-4.0
-1628.0
-1628.0
-27921.0
35.0
35.0
0.0
0.0
-------------------------------------------
-12756.724722845995
-12756.724722845995
0.0
0.0
0.0
0.0
-------------------------------------------
5659.634062432482
5659.634062432482
5656.0
5656.0
5653.0
5653.0
13187.0
-1635.0
-1635.0
27921.0
-35.0
-35.0
0.0
0.0
0.0
0.0
-1665.0
-1665.0
-1661.0
-1661.0
-------------------------------------------
7436.196059314895
7436.196059314895
0.0
0.0
-------------------------------------------
-7194.0
148.0
-5686.0
-5686.0
22259.0
-13215.0
100.0
100.0

In [116]:
for i in range(20):
    [print(f) for f in fehler[wert[i][0]]]
    print("-------------------------------------------")

12756.724722845995
12756.724722845995
0.0
0.0
48.0
48.0
-15491.944657318918
-15491.944657318918
-------------------------------------------
43.57991052826401
43.57991052826401
-24954.271385183318
-24954.271385183318
-62.0
9750.0
4.0
4.0
0.0
0.0
-------------------------------------------
5686.0
5686.0
-22259.0
13215.0
39.57991052826401
39.57991052826401
-1602.0
-1602.0
33.0
33.0
24954.271385183318
24954.271385183318
62.0
-9750.0
-1632.0
-1632.0
0.0
0.0
-4.0
-4.0
-1628.0
-1628.0
-27921.0
35.0
35.0
0.0
0.0
-------------------------------------------
-12756.724722845995
-12756.724722845995
0.0
0.0
0.0
0.0
-------------------------------------------
5659.634062432482
5659.634062432482
5656.0
5656.0
5653.0
5653.0
13187.0
-1635.0
-1635.0
27921.0
-35.0
-35.0
0.0
0.0
0.0
0.0
-1665.0
-1665.0
-1661.0
-1661.0
-------------------------------------------
7436.196059314895
7436.196059314895
0.0
0.0
-------------------------------------------
-7194.0
148.0
-5686.0
-5686.0
22259.0
-13215.0
100.0
100.0

In [91]:
x = [l for l in y.split('Teile')]
for var in x:
    if not var:
        continue
    xx = var.splitlines()
    
    new = []
    for line in xx:
        l = line.split()
        if len(l) > 3:
            print(line)
            continue
        if len(l) < 3:
            continue
            
        l[2] = int(float(l[2]))
        new.append(l)
    new.sort(key = lambda x: x[2])
    print()
    for line in new:
        print(line[0] + '\t' + line[1] + '\t' + str(line[2]))
    print()
    print()

 1497APD auf bei Differenz: 14455.176955343246

1497APD	370APD	-32952
1497APD	1004APD	-32829
1497APD	52APD	-32790
1497APD	47APD	-32787
1497APD	162APD	-32785
1497APD	1806APD	-32772
1497APD	2328APD	-32762
1497APD	1697APD	-32760
1497APD	419APD	-32760
1497APD	558APD	-32759
1497APD	2185APD	-32748
1497APD	339APD	-6667
1497APD	26APD	-6349
1497APD	1028APD	-2769
1497APD	911APD	-2754
1497APD	1961APD	-2469
1497APD	907APD	-2413
1497APD	607APD	-2275
1497APD	417APD	-306
1497APD	163APD	-267
1497APD	660APD	-238
1497APD	1664APD	-237
1497APD	186APD	-209
1497APD	370APD	-190
1497APD	370APD	-190
1497APD	22APD	-174
1497APD	678APD	-153
1497APD	2414APD	-146
1497APD	2414APD	-146
1497APD	8APD	-145
1497APD	234APD	-137
1497APD	2374APD	-136
1497APD	847APD	-136
1497APD	2028APD	-135
1497APD	460APD	-135
1497APD	107APD	-134
1497APD	1129APD	-134
1497APD	1372APD	-134
1497APD	1426APD	-134
1497APD	1832APD	-134
1497APD	2129APD	-134
1497APD	264APD	-134
1497APD	292APD	-134
1497APD	442APD	-134
1497APD	1325APD	-133
1497APD	152

1806APD	1004APD	-62
1806APD	558APD_1	-47
1806APD	558APD_1	-47
1806APD	1697APD	-43
1806APD	1697APD	-43
1806APD	186APD	-36
1806APD	2414APD	-36
1806APD	2414APD	-36
1806APD	1697APD	-30
1806APD	1697APD	-30
1806APD	52APD	-28
1806APD	52APD	-28
1806APD	22APD	-26
1806APD	558APD_2	-26
1806APD	558APD_2	-26
1806APD	52APD	-24
1806APD	52APD	-24
1806APD	674APD	-24
1806APD	674APD	-24
1806APD	2328APD_2	-23
1806APD	2328APD_2	-23
1806APD	108APD	-22
1806APD	108APD	-22
1806APD	419APD	-20
1806APD	419APD	-20
1806APD	678APD	-19
1806APD	164APD	-15
1806APD	164APD	-15
1806APD	370APD	-15
1806APD	370APD	-15
1806APD	2414APD	-14
1806APD	2414APD	-14
1806APD	8APD	-13
1806APD	1004APD	-12
1806APD	1004APD	-12
1806APD	162APD	-10
1806APD	162APD	-10
1806APD	47APD	-7
1806APD	47APD	-7
1806APD	162APD	-6
1806APD	162APD	-6
1806APD	1662APD	-6
1806APD	383APD	-6
1806APD	383APD	-6
1806APD	607APD	-6
1806APD	1013APD	-5
1806APD	1178APD	-5
1806APD	1961APD	-5
1806APD	203APD	-5
1806APD	2101APD	-5
1806APD	2176APD	-5
1806APD	2185APD_1	-5
18

1697APD	163APD	-105
1697APD	1497APD_1	-103
1697APD	1497APD_1	-103
1697APD	660APD	-96
1697APD	1664APD	-67
1697APD	2328APD_1	-63
1697APD	2328APD_1	-63
1697APD	558APD_1	-61
1697APD	558APD_1	-61
1697APD	2185APD_1	-55
1697APD	2185APD_1	-55
1697APD	558APD_1	-38
1697APD	558APD_1	-38
1697APD	1004APD_1	-37
1697APD	1004APD_1	-37
1697APD	1013APD	-35
1697APD	1178APD	-35
1697APD	1662APD	-35
1697APD	1961APD	-35
1697APD	2185APD_1	-35
1697APD	2185APD_1	-35
1697APD	2328APD_1	-35
1697APD	2328APD_1	-35
1697APD	354APD	-35
1697APD	499APD	-35
1697APD	513APD	-35
1697APD	607APD	-35
1697APD	796APD	-35
1697APD	901APD	-35
1697APD	911APD	-35
1697APD	203APD	-34
1697APD	2101APD	-34
1697APD	2176APD	-34
1697APD	2447APD	-34
1697APD	252APD	-34
1697APD	419APD_1	-34
1697APD	419APD_1	-34
1697APD	517APD	-34
1697APD	558APD_1	-34
1697APD	558APD_1	-34
1697APD	66APD	-34
1697APD	710APD	-34
1697APD	804APD	-34
1697APD	1028APD	-33
1697APD	1120APD	-33
1697APD	1190APD	-33
1697APD	1214APD	-33
1697APD	1690APD	-33
1697APD	1790APD	-33
1

47APD	1004APD_1	32670
47APD	1004APD_1	32670


 157APD auf bei Differenz: 9244.343572171534

157APD	216APD	-3145
157APD	1701APD	-3088
157APD	764APD	-3063
157APD	2125APD	-3062
157APD	684APD	-3059
157APD	898APD	-3051
157APD	1966APD	-3037
157APD	1621APD	-3008
157APD	277APD	-1202
157APD	241APD	-1142
157APD	1955APD	-474
157APD	2380APD	-460
157APD	1543APD	-454
157APD	638APD	-453
157APD	1390APD	-440
157APD	1626APD	-433
157APD	88APD	-433
157APD	216APD	-131
157APD	216APD	-131
157APD	593APD	-108
157APD	1156APD	-96
157APD	2296APD	-96
157APD	1701APD	-86
157APD	1701APD	-86
157APD	2380APD	-85
157APD	2380APD	-85
157APD	684APD	-85
157APD	684APD	-85
157APD	2021APD	-83
157APD	1390APD	-82
157APD	1390APD	-82
157APD	1955APD	-82
157APD	1955APD	-82
157APD	1626APD	-74
157APD	1626APD	-74
157APD	764APD	-65
157APD	764APD	-65
157APD	2125APD	-56
157APD	2125APD	-56
157APD	638APD	-56
157APD	638APD	-56
157APD	986APD	-54
157APD	1514APD	-53
157APD	1514APD	-53
157APD	216APD	-45
157APD	216APD	-45
157APD	216APD	-45
157APD	

In [163]:
file = open('apd100.lst')
fehler = {}
for line in file:
    fehler.setdefault(line, 0)
    fehler[line] += 1
ups = {}
ups2 = {}
for w in fehler:
    v = fehler[w]
    a,b,c = w.split()
    c = float(c)
    if c.is_integer():
        ups.setdefault(v, 0)
        ups[v] += 1
    else:
        ups2.setdefault(v, 0)
        ups2[v] += 1

ups,ups2

({2: 75299, 4: 526, 8: 2, 6: 25, 10: 2, 1: 23553}, {2: 1103})

In [11]:
file = open('my_apd100.lst')
fehler = {}
for line in file:
    fehler.setdefault(line, 0)
    fehler[line] += 1
ups = {}
ups2 = {}
for w in fehler:
    v = fehler[w]
    a,b,c = w.split()
    c = float(c)
    if c.is_integer():
        ups.setdefault(v, 0)
        ups[v] += 1
    else:
        ups2.setdefault(v, 0)
        ups2[v] += 1

ups,ups2

({1: 98847, 2: 531, 3: 25, 5: 2, 4: 2}, {1: 1103})

In [182]:
file = open('apd3.lst')
r = {}
for line in file:
    a,b,c = line.split()
    r.setdefault(a, 0)
    r[a] += 1
    r.setdefault(b, 0)
    r[b] += 1

u = [[] for i in range(10)]
for t in r:
    if r[t] < 10:
        u[r[t]].append(t)

In [25]:
x = read_file('mytest1000.lst')

In [33]:
y = [model.addVar(name = a + '\t' + b + '\t'+ str(dist)) for a,b,dist in x]
model.update()

In [38]:
y[0].VarName

'1001APD\t100APD\t53816.0'

In [44]:
a = 'abc_0'

In [45]:
a[:-2]

'abc'

In [124]:
a['b'] = 5
a

{'a': 2, 'b': 5, 'c': 1}

In [122]:
model = Model("Gen")
a = {'a': 2, 'b': 3, 'c': 1} 
X['a'].start = 3
model.update()
model

<gurobi.Model Continuous instance Gen: 0 constrs, 0 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>

In [92]:
X['a'].start,X['a'].Start,X['b'].start,X['b'].Start,X['c'].start,X['c'].Start

(3.0, 3.0, 2.0, 2.0, 1e+101, 1e+101)

In [99]:
model.Start

[]

In [103]:
a.items()

dict_items([('a', 2), ('b', 3), ('c', 1)])

In [107]:
a, b = min( it.product([1,5,19],[9,-6,4]), key = lambda x: abs(x[0]-x[1]-5))

In [108]:
a,b

(1, -6)

In [111]:
print('a',a)

a 1


In [113]:
(1,5) + (4,2)

(1, 5, 4, 2)

In [115]:
def foo():
    Point = coll.namedtuple('Point', ['x', 'y'])
    p = Point(1,5)
    return p
t = foo()
print(t)

Point(x=1, y=5)


In [125]:
a['r'] = 5
a

{'a': 2, 'b': 5, 'c': 1, 'r': 5}

In [118]:
coll.namedtuple('Gruppe', ['min', 'max', 'median', 'anzahl'])

Gruppe(min=1, max=2, median=3, anzahl=4)

In [69]:
mo = Model('A')
v = mo.addVars([1,2,3])
c = [mo.addVar(name = 'a'),mo.addVar(name = 'b'),mo.addVar(name = 'c')]
mo.addConstr(v[1]-v[2]-3 <= c[0])
mo.addConstr(-v[1]+v[2]+3 <= c[0])
k1 = mo.addConstr(v[2]-v[3]-2 <= c[1])
k2 = mo.addConstr(-v[2]+v[3]+2 <= c[1])
mo.addConstr(v[1]-v[3]-6 <= c[2])
mo.addConstr(-v[1]+v[3]+6 <= c[2])
mo.setObjective(c[0]+c[1]+c[2],GRB.MINIMIZE)
mo.optimize()
print(v[1].X,v[2].X,v[3].X)
v[4] = mo.addVar()
mo.remove(c[1])
c[1] = mo.addVar(name = 'd')
mo.addConstr(v[1]-v[4]-6 <= c[1])
mo.addConstr(-v[1]+v[4]+6 <= c[1])
mo.setObjective(c[0]+c[1]+c[2],GRB.MINIMIZE)
mo.optimize()
print(v[1].X,v[2].X,v[3].X,v[4].X)

Optimize a model with 6 rows, 6 columns and 18 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 6e+00]
Presolve time: 0.09s
Presolved: 6 rows, 6 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.100000e+01   0.000000e+00      0s
       4    1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.11 seconds
Optimal objective  1.000000000e+00
6.0 2.0 0.0
Optimize a model with 8 rows, 7 columns and 22 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 6e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.04 seconds
Optimal objective  1.000000000e+00
6.0 2.0 0.0 0.0


In [37]:
mo.getConstrs()

[<gurobi.Constr R0>,
 <gurobi.Constr R1>,
 <gurobi.Constr R2>,
 <gurobi.Constr R3>,
 <gurobi.Constr R4>,
 <gurobi.Constr R5>,
 <gurobi.Constr R6>,
 <gurobi.Constr R7>]

In [38]:
mo.getConstrs()
mo.remove(k1)

In [45]:
mo.getConstrs()

[<gurobi.Constr R0>,
 <gurobi.Constr R1>,
 <gurobi.Constr R3>,
 <gurobi.Constr R4>,
 <gurobi.Constr R5>,
 <gurobi.Constr R6>,
 <gurobi.Constr R7>,
 <gurobi.Constr R8>]

In [44]:
mo.update()

In [61]:
for i in enumerate([10,11,12]):
    print(i)

(0, 10)
(1, 11)
(2, 12)


In [71]:
x = v[1] + v[2]

In [75]:
v[1].X,v[2].X

(6.0, 2.0)

In [72]:
x.getValue()

8.0

In [100]:
mo.getVars()

[<gurobi.Var C0 (value 6.0)>,
 <gurobi.Var C1 (value 2.0)>,
 <gurobi.Var C2 (value 0.0)>,
 <gurobi.Var a (value 1.0)>,
 <gurobi.Var c (value 0.0)>,
 <gurobi.Var C6 (value 0.0)>,
 <gurobi.Var d (value 0.0)>]

In [32]:
i = {'c':2,'a':3,'b':1}
sorted(i.items(), key = lambda x: x[1])

[('b', 1), ('c', 2), ('a', 3)]

In [ ]:
import mat